This demo is a implementation of capsule network according to Dynamic Routing Between Capsules(https://arxiv.org/pdf/1710.09829.pdf), the main ideas:
- Use capsule vectors to replace scalars of each layer, for example, in a conventional neuron, the input are scalars whereas in a capsule, the input are capsule vectors.
- Use coupling coefficients to replace weights between two layers. The connection between neurons of previous layer and neurons of current layer are replaced by dynamic routing algorithms.
- Use the vector length to show how likely it exists. 

The code is referred to github.com/naturomics and my work includes:
- Map the object of capsule network into an explicit procedures of implementing capsule model.
- Add comments based on my personal understanding.

In [1]:
#Import required packages
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
#Load MNIST dataset
import tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [69]:
print(len(mnist.train.images))

55000


In [70]:
print(len(mnist.test.images))

10000


First, define a squash function to get the length of a vector which stands for the existence probability, a vector which has a large norm value means high probability.

In [71]:
batch_size = 50
epsilon = 1e-9
#This function referred to https://github.com/naturomics/CapsNet-Tensorflow
def squash(vector):

    '''Squashing function corresponding to Eq. 1

    Args:

        vector: A tensor with shape [batch_size, 1, num_caps, vec_len, 1] or [batch_size, num_caps, vec_len, 1].

    Returns:

        A tensor with the same shape as vector but squashed in 'vec_len' dimension.

    '''

    vec_squared_norm = tf.reduce_sum(tf.square(vector), -2, keep_dims=True)

    scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + epsilon)

    vec_squashed = scalar_factor * vector  # element-wise

    return(vec_squashed)

For the dynamic routing algorithm, it is very different from conventional neural network, in previous neural network models, the values of the next layers are determined by the output of previous layers and the weights between these two layers, the weights are trainable parameters. **However, in a capsule network, the relationship between two layers are determined by dynamic routing mechanism, the coefficients are calculated by iterations.** Actually, according to this [blog](http://www.sohu.com/a/218609089_500659), the dynamic routing is a way of finding clustering center for previous capsules. The graph is referred to http://www.sohu.com/a/218609089_500659.
![dynamic_routing](http://5b0988e595225.cdn.sohucs.com/images/20180124/7fa017c7daf94e309971d80f1f1e3832.jpeg)

Note, the affline mapping weights are shared across capsules of one layer like this way:
![dynamic](http://5b0988e595225.cdn.sohucs.com/images/20180124/fa268f22122f44518f10cd831cc7c04c.jpeg)

In [72]:
#This function referred to https://github.com/naturomics/CapsNet-Tensorflow
def routing(input, b_IJ, iter_routing=10):

    ''' The routing algorithm.
    Args:

        input: A Tensor with [batch_size, num_caps_l=1152, 1, length(u_i)=8, 1]

               shape, num_caps_l meaning the number of capsule in the layer l.

    Returns:

        A Tensor of shape [batch_size, num_caps_l_plus_1, length(v_j)=16, 1]

        representing the vector output `v_j` in the layer l+1

    Notes:

        u_i represents the vector output of capsule i in the layer l, and

        v_j the vector output of capsule j in the layer l+1.

     '''
    # W: [1, num_caps_i, num_caps_j * len_v_j, len_u_j, 1]
    #Note, these weights are shared across different capsules within one layer

    W = tf.get_variable('Weight', shape=(1, 1152, 160, 8, 1), dtype=tf.float32,

                        initializer=tf.random_normal_initializer(0.01))

    biases = tf.get_variable('bias', shape=(1, 1, 10, 16, 1))

    # Eq.2, calc u_hat

    # Since tf.matmul is a time-consuming op,

    # A better solution is using element-wise multiply, reduce_sum and reshape

    # ops instead. Matmul [a, b] x [b, c] is equal to a series ops as

    # element-wise multiply [a*c, b] * [a*c, b], reduce_sum at axis=1 and

    # reshape to [a, c]

    input = tf.tile(input, [1, 1, 160, 1, 1])

    assert input.get_shape() == [batch_size, 1152, 160, 8, 1]

    u_hat = tf.reduce_sum(W * input, axis=3, keep_dims=True)

    u_hat = tf.reshape(u_hat, shape=[-1, 1152, 10, 16, 1])

    assert u_hat.get_shape() == [batch_size, 1152, 10, 16, 1]


    # In forward, u_hat_stopped = u_hat; in backward, no gradient passed back from u_hat_stopped to u_hat
    u_hat_stopped = tf.stop_gradient(u_hat, name='stop_gradient')

    # line 3,for r iterations do
    for r_iter in range(iter_routing):

        with tf.variable_scope('iter_' + str(r_iter)):

            # line 4:

            # => [batch_size, 1152, 10, 1, 1]
            #print(b_IJ)

            c_IJ = tf.nn.softmax(b_IJ, dim=2)



            # At last iteration, use `u_hat` in order to receive gradients from the following graph

            if r_iter == iter_routing - 1:

                # line 5:

                # weighting u_hat with c_IJ, element-wise in the last two dims

                # => [batch_size, 1152, 10, 16, 1]

                s_J = tf.multiply(c_IJ, u_hat)

                # then sum in the second dim, resulting in [batch_size, 1, 10, 16, 1]

                s_J = tf.reduce_sum(s_J, axis=1, keep_dims=True) + biases

                assert s_J.get_shape() == [batch_size, 1, 10, 16, 1]



                # line 6:

                # squash using Eq.1,

                v_J = squash(s_J)

                assert v_J.get_shape() == [batch_size, 1, 10, 16, 1]

            elif r_iter < iter_routing - 1:  # Inner iterations, do not apply backpropagation

                s_J = tf.multiply(c_IJ, u_hat_stopped)

                s_J = tf.reduce_sum(s_J, axis=1, keep_dims=True) + biases

                v_J = squash(s_J)


                # line 7:

                # reshape & tile v_j from [batch_size ,1, 10, 16, 1] to [batch_size, 1152, 10, 16, 1]

                # then matmul in the last tow dim: [16, 1].T x [16, 1] => [1, 1], reduce mean in the

                # batch_size dim, resulting in [1, 1152, 10, 1, 1]

                v_J_tiled = tf.tile(v_J, [1, 1152, 1, 1, 1])

                u_produce_v = tf.reduce_sum(u_hat_stopped * v_J_tiled, axis=3, keep_dims=True)

                assert u_produce_v.get_shape() == [batch_size, 1152, 10, 1, 1]


                # b_IJ += tf.reduce_sum(u_produce_v, axis=0, keep_dims=True)

                b_IJ += u_produce_v



    return(v_J)

**Note, there are no activation functions and pooling methods, I think this capsule model tries to make full use of information instead of filtering them out.** In this example there are three layers in sequence:
- 1 A 9X9X256 conv filter to get image features 20X20X256
- 2 A group(8) of 9X9X32 conv filter to get 1152 capsules with lengths of 8, squashing function is implemented for the output
- 3 10 capsules with lengths of 16

There are 10 capsules in the output, each has a length of 16.

In [73]:
graph_capsule = tf.Graph()
with graph_capsule.as_default() as g:
    #Create input placeholders
    x = tf.placeholder(tf.float32, [batch_size, 784])
    y_ = tf.placeholder(tf.float32, [batch_size,10])
    #Reshape
    x_image = tf.reshape(x, [-1,28,28,1])
    #Use a convnet to extract local features as input for capsules
    with tf.name_scope('conv_input'):
        #batch_size, 20, 20, 256
        input_data = tf.contrib.layers.conv2d(inputs=x_image, num_outputs=256, 
                                              kernel_size=9, padding='valid')
        
    #PrimaryCaps
    ######################Create first capsule layer#################################
    capsules = []
    #The vector length of the capsule is 8
    #Create 8 conv outputs, stacks of 8 scales
    for i in range(8):
        with tf.name_scope('cap_conv_' + str(i)):
            #output: batch_size, 6, 6, 32
            output = tf.contrib.layers.conv2d(inputs=input_data, num_outputs=32, 
                                              kernel_size=9, stride=2, padding='valid')
            output = tf.reshape(output, [batch_size, -1, 1, 1])
            capsules.append(output)
    #Concatenate the outputs to form capsule vectors
    #1152 capsule vectors of 8-length for each image
    capsules_0 = tf.concat(capsules, axis=2)
    #remove the last 1 dim, batch_size*1152*8
    #Normalize the vectors by squashing function
    capsules_0 = squash(capsules_0)
    
    
    #####################Create second capsule layer########################################
    #print(capsules_0)
    #Routing,DIgitCaps
    #Output 10 capsules of 16-length
    with tf.name_scope('routing'):
        #Batch_size*1152*1*8*1, extend dimension? Not quite sure the reasons behind
        outputs = tf.reshape(capsules_0, shape=(batch_size, -1, 1, 
                                              capsules_0.shape[-2].value, 1))
        print(outputs)
        #Initializing coefficients
        b_IJ = tf.constant(np.zeros([1, capsules_0.shape[1].value, 10, 1, 1], dtype=np.float32))
        capsules_1 = routing(outputs, b_IJ)
    #10 capsules, each capsule consist of 16-length
    capsules_1 = tf.squeeze(capsules_1, axis=1)
    print(capsules_1)
    

Tensor("routing/Reshape:0", shape=(50, 1152, 1, 8, 1), dtype=float32)
Tensor("Squeeze:0", shape=(50, 10, 16, 1), dtype=float32)


**Unlike previous CNNs, RNNs, where we use fully-connected network to map features into a output layer whose neurons corresponding to the classes, here we use the norms of each capsule to represent the probabilty of its existence.** The number of capsules is corresponding the number of classes, and the capsule which has the longest norm stands for the prediction.

In [81]:
epsilon = 1e-9
with graph_capsule.as_default() as g:
    with tf.variable_scope('Masking'):
        #Calculate the norms of each capsule
        v_length = tf.sqrt(tf.reduce_sum(tf.square(capsules_1),
                                           axis=2, keep_dims=True) + epsilon)
        #Use softmax to get prediction probabilities
        softmax_v = tf.nn.softmax(v_length, dim=1)
        assert softmax_v.get_shape() == [batch_size, 10, 1, 1]

        # b). pick out the index of max softmax val of the 10 caps
        # [batch_size, 10, 1, 1] => [batch_size] (index)
        argmax_idx = tf.to_int32(tf.argmax(softmax_v, axis=1))
        assert argmax_idx.get_shape() == [batch_size, 1, 1]
        
        argmax_idx = tf.reshape(argmax_idx, shape=(batch_size, ))
        #Calculate accuracy
        label_idx = tf.to_int32(tf.argmax(y_, axis=1))
        correct_bool = tf.cast(tf.equal(argmax_idx, label_idx), tf.float32)
        accuracy = tf.reduce_mean(correct_bool)
        correct_num = tf.reduce_sum(correct_bool)
        #print(correct_num)
        
        #Select the corresponding capsule of 16-length
        masked_v = tf.multiply(tf.squeeze(capsules_1), tf.reshape(y_, (-1, 10, 1)))
        print(masked_v)
        #v_length = tf.sqrt(tf.reduce_sum(tf.square(capsules_1), axis=2, keep_dims=True) + epsilon)

Tensor("Masking_1/Sum_1:0", shape=(), dtype=float32)
Tensor("Masking_1/Mul:0", shape=(50, 10, 16), dtype=float32)


**In order to validate how well the capsules can retain the original information, in the paper, Hinton tried to reconstruct the original images based on the output capsules.** There three fully-connected layers:
- A layer 160X512
- A layer 512X1024
- A layer 1024X784

For this part, it is a regression problem.

In [84]:
with graph_capsule.as_default() as g:
    ###########Decoder, to reconstruct the digits#############
    with tf.name_scope('Decoder'):
        #Use the corresponding capsule vector to reconstruct the digit
        #Use two fully-connected layers
        vector_j = tf.reshape(masked_v, shape=(batch_size, -1))
        print(vector_j)
        fc1 = tf.contrib.layers.fully_connected(vector_j, num_outputs=512)
        assert fc1.get_shape() == [batch_size, 512]
           
        fc2 = tf.contrib.layers.fully_connected(fc1, num_outputs=1024)
        assert fc2.get_shape() == [batch_size, 1024]

        decoded = tf.contrib.layers.fully_connected(fc2, num_outputs=784, activation_fn=tf.sigmoid)
        print(decoded)

Tensor("Decoder_1/Reshape:0", shape=(50, 160), dtype=float32)
Tensor("Decoder_1/fully_connected_2/Sigmoid:0", shape=(50, 784), dtype=float32)


**Now it's time for loss functions, unlike previous cross-entropy loss, here a marginal loss is adopted, which includes two parts**:
- The loss for the longest capsule, the norm needs to be as large as 0.9
- The loss for the shorter capsules, the norm needs to be smaller than 0.1
The purpose of the loss function is to make the dominant capsule as strong as possible whereas the other capsules less important.

In addtion, a regularization loss which means the construction error is also considered. The loss is Mean Squared Error.

In [76]:
#Define Loss
lambda_value = 0.5
regularization_scale = 0.0005
with graph_capsule.as_default() as g:
    #####################Define Margin Loss ##################
    #The loss for the prediction of dominant capsule
    max_l = tf.square(tf.maximum(0.,0.9 - v_length))
    #The loss for prediction of other capsule
    max_r = tf.square(tf.maximum(0., v_length - 0.1))
    assert max_l.get_shape() == [batch_size, 10, 1, 1]

    # reshape: [batch_size, 10, 1, 1] => [batch_size, 10]
    max_l = tf.reshape(max_l, shape=(batch_size, -1))
    max_r = tf.reshape(max_r, shape=(batch_size, -1))
    # calc T_c: [batch_size, 10]
    # T_c = Y, is my understanding correct? Try it.
    T_c = y_
    # [batch_size, 10], element-wise multiply
    L_c = T_c * max_l + lambda_value * (1 - T_c) * max_r
    margin_loss = tf.reduce_mean(tf.reduce_sum(L_c, axis=1))
    
    ####################Define Reconstruction Loss############
    #mean square error for the regression
    orgin = tf.reshape(x, shape=(batch_size, -1))
    squared = tf.square(decoded - orgin)
    reconstruction_err = tf.reduce_mean(squared)
    # Total loss
    total_loss = margin_loss + regularization_scale * reconstruction_err
    print(total_loss)

Tensor("add_3:0", shape=(), dtype=float32)


In [77]:
#Define Optimization
with graph_capsule.as_default() as g:
    optimizer = tf.train.AdamOptimizer(0.0001)
    train_op = optimizer.minimize(total_loss)

In [83]:
epochs = 3
num_steps = int(len(mnist.train.images)/batch_size)
with tf.Session(graph=graph_capsule) as sess:
    #Initialize variables
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    for _ in range(epochs):
        for step in range(num_steps):
            batch_data, batch_labels = mnist.train.next_batch(batch_size)
            feed_dict = {x : batch_data, y_ : batch_labels}
            #Train
            _, loss, acc = sess.run([train_op, total_loss, accuracy], feed_dict=feed_dict)
            if step%500 == 0:
                print('Loss:', loss, acc)
     
    loops = int(len(mnist.test.images)/batch_size)
    count = 0
    for i in range(loops):
        batch_data, batch_labels = mnist.test.next_batch(batch_size)
        feed_dict = {x : batch_data, y_ : batch_labels}
        #Train
        num = sess.run(correct_num, feed_dict=feed_dict)
        count += num
    print('Testing Accuracy: ', count*1.0/10000)
    

Loss: 0.555879 0.04
Loss: 0.0449784 0.96
Loss: 0.0336064 1.0
Loss: 0.0103637 1.0
Loss: 0.0118929 1.0
Loss: 0.00713452 1.0
Loss: 0.00686674 1.0
Loss: 0.0133592 1.0
Loss: 0.00595833 1.0
Testing Accuracy:  0.9866


# References
1. Dynamic Routing Between Capsules, Sara Sabour and etc, NIPS 2017.
2. 揭开迷雾，来一顿美味的「Capsule」盛宴, 苏剑林,  http://www.sohu.com/a/218609089_500659 .
3. 从K-Means到Capsule, 苏剑林, https://kexue.fm/archives/5112
4. Capsule Tensorflow Implementation, Naturomics, https://github.com/naturomics/CapsNet-Tensorflow